# LSTM com Embeddings (Feat. Machado de Assis)

In [44]:
import collections
import json
import re
from typing import List, Set, Dict, Tuple, Generator

import numpy as np
import pandas as pd
import scipy
import tensorflow as tf

CAMINHO_MODELO = "LSTM-de-assis.h5"
CAMINHO_DICIONARIO = "dicionario.json"
CAMINHO_DICIONARIO_INDICES = "dicionario_indices.json"
EMBEDDING_UNITS = 128
LSTM_UNITS = 128

In [62]:
def criar_dicionario(palavras: List[str], minimo_palavras: int) -> (Dict[str, int], Dict[int, str], Dict[str, int]):
    dicionario_freq = collections.Counter(palavras)
    palavras_no_dicionario = [palavra for palavra, qtd in dicionario_freq.items() if qtd > minimo_palavras]
    dicionario = {palavra: indice for (indice, palavra) in enumerate(palavras_no_dicionario)}
    dicionario["UNK"] = len(dicionario)
    dicionario_indices = dict(zip(dicionario.values(), dicionario.keys()))
    return dicionario, dicionario_indices, dicionario_freq

def criar_dataset(minimo_palavras_frase=3, minimo_palavras_dicionario = 2):
    df_obras = pd.read_csv("./obras_machado_de_assis.csv")
    df_obras = df_obras[df_obras["categoria"] != "tradução"]


    dataset = pd.Series(np.concatenate(df_obras["texto"]\
                .str.replace("\n+", " ")\
                .str.replace("\.+", ".")\
                .str.replace("\;", ",")\
                .str.split("([\.\?!][\'\"\u2018\u2019\u201c\u201d\)\]]*\s*(?<!\w\.\w.)(?<![A-Z][a-z][a-z]\.)(?<![A-Z][a-z]\.)(?<![A-Z]\.)\s+)")))\
                .str.strip()
    df_dataset = pd.DataFrame(list(zip(dataset, dataset[1:])))
    df_dataset = df_dataset[df_dataset[0].str.strip().apply(lambda row: len(row.split())) != 1]
    dataset = (df_dataset[0]+df_dataset[1]).reset_index(drop=True)
    dataset = dataset\
        .str.replace(" *\. *", " . ")\
        .str.replace(" *\, *", " , ")\
        .str.replace(" *\! *", " ! ")\
        .str.replace(" *\? *", " ? ")\
        .str.replace("[^\w\s\d\.\,\?\!]", "")\
        .str.strip()\
        .str.lower()
    dataset = dataset[dataset.apply(lambda row: len(row.split())) > minimo_palavras_frase].reset_index(drop=True)
    palavras = np.concatenate(dataset.apply(lambda row: row.split()))

    dicionario, dicionario_indices, dicionario_freq = criar_dicionario(palavras, minimo_palavras_dicionario)
    dataset = dataset.apply(lambda row: [dicionario.get(palavra, len(dicionario)-1) for palavra in row.split()])
    return dataset, dicionario, dicionario_indices, dicionario_freq

def separar_dataset(dataset):
    indices_test = np.random.randint(0, tamanho_dataset, tamanho_dataset//100)
    test_dataset = dataset[indices_test].reset_index(drop=True)
    mask = np.ones(tamanho_dataset, dtype=bool)
    mask[indices_test] = False
    dataset = dataset[mask].reset_index(drop=True)
    return dataset, test_dataset

dataset, dicionario, dicionario_indices, dicionario_freq = criar_dataset()
tamanho_dicionario = len(dicionario)
tamanho_dataset = len(dataset)

dataset, test_dataset = separar_dataset(dataset)

with open(CAMINHO_DICIONARIO, "w") as fp:
    json.dump(dicionario, fp)
with open(CAMINHO_DICIONARIO_INDICES, "w") as fp:
    json.dump(dicionario_indices, fp)

In [58]:
def criar_modelo(dim_entrada, dim_embedding, dim_lstm):
    model = tf.keras.models.Sequential(name="LSTM-de-Assis")
    model.add(tf.keras.layers.Input((None,), name="entrada", batch_size=1))
    model.add(tf.keras.layers.Embedding(dim_entrada, dim_embedding, name='embedding'))
    model.add(tf.keras.layers.LSTM(dim_lstm, name="lstm1", activation="tanh", return_sequences=True))
    model.add(tf.keras.layers.LSTM(dim_lstm, name="lstm2", activation="tanh", return_sequences=True))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dim_lstm, activation="tanh"), name="densa1"))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dim_entrada, activation="softmax"), name="densa2"))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    return model


modelo = criar_modelo(tamanho_dicionario, EMBEDDING_UNITS, LSTM_UNITS)
modelo.summary()

Model: "LSTM-de-Assis"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 128)            3635072   
_________________________________________________________________
lstm1 (LSTM)                 (1, None, 128)            131584    
_________________________________________________________________
lstm2 (LSTM)                 (1, None, 128)            131584    
_________________________________________________________________
densa1 (TimeDistributed)     (None, None, 128)         16512     
_________________________________________________________________
densa2 (TimeDistributed)     (None, None, 28399)       3663471   
Total params: 7,578,223
Trainable params: 7,578,223
Non-trainable params: 0
_________________________________________________________________


In [12]:
def criar_gerador(dataset):
    while 42:
        indices = np.arange(len(dataset))
        np.random.shuffle(indices)
        for i in indices:
            yield np.array(dataset[i][:-1]).reshape(1, -1), np.array(dataset[i][1:]).reshape(1, -1, 1)

gerador = criar_gerador(dataset)
gerador_val = criar_gerador(test_dataset)
history = modelo.fit_generator(gerador, epochs=50, steps_per_epoch=tamanho_dataset//10, validation_data=gerador_val, validation_steps=len(test_dataset), validation_freq=1)
model.save('LSTM-de-assis.h5')

Epoch 1/50
10181/10181 [==============================] - 645s 63ms/step - loss: 6.5426 - accuracy: 0.0916 - val_loss: 6.1982 - val_accuracy: 0.1098
Epoch 2/50
10181/10181 [==============================] - 744s 73ms/step - loss: 6.0071 - accuracy: 0.1141 - val_loss: 5.9623 - val_accuracy: 0.1198
Epoch 3/50
10181/10181 [==============================] - 643s 63ms/step - loss: 5.8661 - accuracy: 0.1256 - val_loss: 5.8698 - val_accuracy: 0.1269
Epoch 4/50
10181/10181 [==============================] - 638s 63ms/step - loss: 5.7849 - accuracy: 0.1336 - val_loss: 5.7779 - val_accuracy: 0.1323
Epoch 5/50
10181/10181 [==============================] - 642s 63ms/step - loss: 5.7594 - accuracy: 0.1377 - val_loss: 5.7671 - val_accuracy: 0.1386
Epoch 6/50
10181/10181 [==============================] - 567s 56ms/step - loss: 5.6964 - accuracy: 0.1417 - val_loss: 5.7345 - val_accuracy: 0.1371
Epoch 7/50
10181/10181 [==============================] - 550s 54ms/step - loss: 5.6604 - accuracy: 0.1470

In [3]:
# modelo = tf.keras.models.load_model(CAMINHO_MODELO)

In [63]:
def gerar_tabela_pred(pred, dicionario_indices):
    pred_df = pd.Series(pred[0][-1])
    pred_df = pd.DataFrame([(dicionario_indices[indice], valor) for indice, valor in pred_df.sort_values(ascending=False).iteritems()])
    pred_df = pred_df.rename(columns={0: "palavra", 1: "probabilidade"})
    return pred_df

def predizer_palavra(modelo, frase, dicionario, dicionario_indices):
    vetor = [[[dicionario.get(palavra, len(dicionario)-1) for palavra in frase.split()]]]
    pred = modelo.predict(vetor)
    tabela = gerar_tabela_pred(pred, dicionario_indices)
    palavra = "UNK"
    pontuacao =  [".", ",", "!", "?"]
    while palavra == "UNK" or (frase[-1] in pontuacao and palavra in pontuacao):
        palavra = np.random.choice(tabela["palavra"], p=tabela["probabilidade"]/tabela["probabilidade"].sum())
    return palavra
    
def criar_frase(modelo, dicionario, dicionario_indices, frase_inicial="", qtd_palavras=10):
    frase = np.random.choice(list(dicionario.keys())) if frase_inicial=="" else frase_inicial.lower()
    while len(frase.split()) < qtd_palavras:
        frase += " " + predizer_palavra(modelo, frase, dicionario, dicionario_indices)
    return frase

for i in range(10):
    print(criar_frase(model, dicionario, dicionario_indices, frase_inicial="", qtd_palavras=10))

nota então olha semana para viver . outro teatro ,
vagido de justiça ! pelo banco . camilo , como
adequada aos joão de melo , demais , não prazer
internúncio , velho elas , meus e adelaide , lhe
bungo e senhoras , quando eu disse escrito , forma
abolido os seus fins . olhos . em casa da
respondeulhes vasconcelos , compôs fora hoje os políticos . de
iras da glória . criança mesmo , página da mãos
cornélia bastava de a ira amigo do o ponta do
apóstrofe do sol ! dos dizer quanto por ocasião paciência
